In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import scipy
import pandas as pd
import numpy as np
import keras

In [ ]:
train_drug = pd.read_csv("/kaggle/input/lish-moa/train_drug.csv")
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_targets_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
train_targets_nonscored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")
test_features = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
sample_submission = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")

In [ ]:
def preprocess(df):
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()

    cp_types = ('trt_cp','ctl_vehicle')
    cp_df = pd.DataFrame(cp_types, columns=['cp_type'])
    # generate binary values using get_dummies
    dum_df = pd.get_dummies(cp_df, columns=["cp_type"], prefix=["cp_type"] )
    # merge with main df bridge_df on key values
    cp_df = cp_df.join(dum_df)
    cp_df = cp_df.set_index(['cp_type'])

    # One-hot encode the cp type col
    df[['cp_type_ctl_vehicle', 'cp_type_trt_cp']] = df.cp_type.apply(lambda x: cp_df.loc[x])
    
    # Label encode cp_time and cp_dose
    df['cp_time_cat'] = le.fit_transform(df['cp_time'])
    df['cp_dose_cat'] = le.fit_transform(df['cp_dose'])
    
    # Set index and drop unnecessary cols
    df = df.drop(['cp_type', 'cp_time', 'cp_dose'], axis=1)
    df = df.set_index('sig_id')

    feature_cols = df.columns[:-4]
    df[feature_cols] = df[feature_cols]/10.0
    
    return df

In [ ]:
clean_train_features = preprocess(train_features)
clean_train_features.head()

In [ ]:
train_targets_scored.head()
train_targets_scored.set_index('sig_id', inplace=True)

In [ ]:
X = clean_train_features
Y = train_targets_scored.astype('float32')

In [ ]:
import keras.backend as K
import math
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
input_shape = X.shape[1]
print(input_shape)
num_examples = X.shape[0]
print(num_examples)
num_classes = len(Y.columns)
print(num_classes)

In [ ]:
model = Sequential()
model.add(Dense(num_examples, activation=tf.nn.relu))
model.add(Dense(256, activation=tf.nn.relu))
model.add(Dense(num_classes, activation=tf.nn.sigmoid))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy', # Call the loss function with the selected layer
              metrics=['accuracy'])

In [ ]:
model.fit(X, Y, batch_size = 32, epochs = 10, validation_split=0.2)

In [ ]:
clean_test_data = preprocess(test_features)

In [ ]:
res = model.predict(clean_test_data)
res_df = pd.DataFrame(res, columns=train_targets_scored.columns)
res_df.insert(0, 'sig_id', test_features.sig_id)
res_df.head()

In [ ]:
res_df.to_csv('/kaggle/working/submission.csv', index=False)